Schritt 1: Beginne hier. Bitte stelle vor der Ausführung des Skripts sicher, dass du über GPU auf dieser Landschaft verfügst (Runtime->Change runtime type->GPU). Für die Verarbeitung der Videos kann leider noch keine TPU genutzt werden, da OpenPose diese nicht unterstützt. In diesem Schritt wird OpenPose auf der Landschaft installiert und erstellt. Dieser Schritt benötigt einige Minuten und muss zu Beginn jeder Session erneut durchgeführt werden (nach jedem Start dieses Google Colab Notebooks). Es ist allerdings natürlich möglich mehrere Videos in einer Sessions zu verarbeiten. Um den Installationsschritt zu starten, drücke einfach auf den "Ausführen"-Button. Für die Installation musst keinen Code anpassen.


In [ ]:
#installs OpenPose (do not edit...thank you to OpenPose user miu200521358 for providing this section of the code!)
# Install OpenPose
! nvcc --version
! nvidia-smi
ver_openpose = "v1.6.0"
! cmake --version
! wget -c "https://github.com/Kitware/CMake/releases/download/v3.17.2/cmake-3.17.2.tar.gz"
! tar xf cmake-3.17.2.tar.gz
! cd cmake-3.17.2 && ./configure && make && sudo make install
# Install libraries
# Basic
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade numpy protobuf opencv-python
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev
# Python dependencies
!pip install -q youtube-dl
# Clone openpose
! git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git 

In [ ]:
# Build openpose
! cd openpose && mkdir build && cd build
! cd openpose/build && cmake ..
! cd openpose/build && make -j`nproc`

Schritt 2: In diesem Schritt verbindest du dich mit deine Google Drive, um auf deine Eingabevideos zuzugreifen. Nachdem du auf den "Ausführen"-Button gegeklickt hast, öffne bitte den angezeigten Link und folge den beschriebenden Schritten, um einen Autorisierung-Code zu erhalten. Kopiere diese bitte in das Eingabefeld, um auf deine Google Drive Daten zuzugreifen.

**Wichtig:** Wenn es nicht erlaubt ist, die Videos online zu speichern und zu bearbeiten, muss OpenPose lokal auf einem Computer installiert werden. Für eine Anleitung hierzu, folge bitte den Schritte unter: https://github.com/CMU-Perceptual-Computing-Lab/openpose

In [ ]:
# importiere google drive, um auf die Videos zuzugreifen (keine Anpassung notwendig)
from google.colab import drive
drive.mount('/content/gdrive')

Step 2.3. Run this step only if you need to change the resolution of your video (e.g., your video is being analyzed sideways).

In [ ]:
#changes resolution of the video
#edit the two paths that include ***YourVideoName*** (be sure to include the file type (e.g., .avi, .mov) after the video name in both locations)
import moviepy.editor as mp
clip = mp.VideoFileClip("/content/gdrive/My Drive/YourVideoName.filetype")
clip_resized = clip.resize(newsize=(1080,1920))
clip_resized.write_videofile("/content/gdrive/My Drive/YourVideoName_resized.filetype")

Schritt 3: Bitt stelle sicher, dass das Video, welches du verarbeiten möchtest auf Google Drive abgelegt wurde. Es ist auch möglich den JSON und Video Ausgabepfad anzupassen (der standardmäßige Pfad ist der Google Drive Hauptfolder)

Vergesse nicht die beiden Platzhalter in dem Code mit deinem VideoName.filtetype zu ersetzten.


In [ ]:
#analysieren das auf Google Drive abgelegte Video mit Openpose
#bearbeite die beiden Pfade, die ***VideoName*** enthalten (vergessen nicht die Dateiendung (z.B. .avi oder .mp4) bei beiden Pfaden hinzuzufügen)
!cd openpose && ./build/examples/openpose/openpose.bin --video /content/gdrive/My\ Drive/***VideoName.filetype*** --number_people_max 1 --write_json /content/gdrive/My\ Drive/ --display 0 --write_video /content/gdrive/My\ Drive/***VideoName_labeled.filetype***

/bin/bash: ./build/examples/openpose/openpose.bin: No such file or directory


Schritt 4: Wenn du mehrere Videos aus einem Ordner gleichzeitig verarbeiten möchtest, kannst du diesen Code ausführen. Es werden alle Videos, die sich in dem angegebenen Ordner befinden, mit OpenPose analysisert und die jeweiligen JSON Dateien erzeugt. Dies ist vorallem sinnvoll, wenn eine große Menge an Videos mit OpenPose analysiert werden müssen.

In [ ]:
import os

files = os.listdir("/content/gdrive/My Drive/Videos")

for file in files:
  !cd openpose && ./build/examples/openpose/openpose.bin --video /content/gdrive/My\ Drive/Videos/$file --number_people_max 1 --write_json /content/gdrive/My\ Drive/Keypoints/$file/ --display 0 --render_pose 0

Wenn OpenPose die Videos erfolgreich analysiert hat, findest du die Ordner mit den JSON Dateien auf deinem Google Drive. Du kannst diese nun runterladen, damit diese weiter verarbeitet und klassifiziert werden können.